In [2]:
from shapely.geometry import shape, box
from gbdxtools import Interface
gbdx = Interface()

query = "ingest_source:GBDX_Workflow AND item_type:GBDX_Task_Output AND attributes.workflow:4609932187205404848"
wkt = box(-180, -90, 180, 90).wkt
features = gbdx.vectors.query(wkt, query, index=None)

In [3]:
from shapely.ops import cascaded_union
union = cascaded_union([shape(f['geometry']) for f in features])
lon, lat = union.centroid.coords[0]

In [4]:
import os
from jupbox import Map, GeojsonLayer, RasterLayer

def render(feature):
    m = Map(center=[lon, lat], api_key=os.environ['MBTOKEN'])
    print id(m)

    idaho_id = feature['properties']['attributes']['idaho_id']
    url = 'http://idaho.geobigdata.io/v1/tile/idaho-images/' + idaho_id + '/{z}/{x}/{y}?'
    url += 'bands=4,2,1&gamma=1.3&highCutoff=0.98&lowCutoff=0.02&brightness=1.0&contrast=1.0&panId=undefined'
    url += '&token=' + gbdx.gbdx_connection.access_token

    tiles = RasterLayer('image', url)
    m.add_layer(tiles)

    lines = GeojsonLayer('water_lines', [feature], 'line', paint={"line-color": '#f50000'})
    m.add_layer(lines)

In [5]:
render(features[3])

4488455504
